In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(1437, with_units=False)
Hist_table


Months,Entries
IntegerAndSexagesimal,Historical
31 ;,"01s 18 ; 28,06"
59 ;,"01s 27 ; 24,16"
90 ;,"03s 15 ; 52,32"
120 ;,"04s 21 ; 10,03"
151 ;,"06s 09 ; 38,08"
181 ;,"07s 14 ; 55,39"
212 ;,"10s 03 ; 23,44"
243 ;,"10s 21 ; 51,50"
273 ;,"11s 27 ; 09,21"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)


In [10]:
# mean motion parameter from the Parisian Alfonsine tables
daily_mean=Historical(Sexagesimal("13;10,35,1,15,11,4,35"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),7)

for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=3):
        Test_recomp[i][1]=daily_mean*Hist_table[i][0]%REVO
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*Test_recomp[i][1]
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])*60

In [11]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(526490.0, nan)

In [12]:
B.astype(float).style.background_gradient(axis=None)

,Entries
Months,
31 ;,0.000000
59 ;,-600.000000
90 ;,0.000000
120 ;,60.000000
151 ;,0.000000
181 ;,60.000000
212 ;,6480000.000000
243 ;,60.000000
273 ;,60.000000


In [13]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Lunar mean months.csv')
B.to_csv('Lunar mean months.zip', index=False, compression=compression_opts)